# Embedding NN on Tokenized sequences

The aim of this week is using **Embeddings** to establish meaning from them in a mathematical way after the **Tokenizer**. Words were mapped to vectors in higher dimensional space, and the semantics of the words then learned when those words were labelled with similar meaning. 

So, for example, when looking at movie reviews, those movies with positive sentiment had the dimensionality of their words ending up ‘pointing’ a particular way, and those with negative sentiment pointing in a different direction. From this, the words in future sentences could have their ‘direction’ established, and from this the sentiment inferred.

The data used is imported from built-in library of **IMDB-review** 

In [0]:
########################################### Check the environment for IPython ###########################################
# NOTE: PLEASE MAKE SURE YOU ARE RUNNING THIS IN A PYTHON3 ENVIRONMENT
import tensorflow as tf
print(tf.__version__)

# This is needed for the iterator over the data
# But not necessary if you have TF 2.0 installed
#!pip install tensorflow==2.0.0-beta0

tf.enable_eager_execution()
# !pip install -q tensorflow-datasets

1.15.0


In [0]:
########################################### Import the dataset from tf ###########################################
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/0.1.0. Subsequent calls will reuse this data.


In [0]:
########################################### Pre-process the training & testing data  ###########################################
import numpy as np
train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []
testing_sentences = []
testing_labels = []

# str(s.tonumpy()) is needed in Python3 instead of just s.numpy()
for s,l in train_data:                                # s&l are tensors, use numpy to take the values         
  training_sentences.append(str(s.numpy()))
  training_labels.append(l.numpy())
  
for s,l in test_data:
  testing_sentences.append(str(s.numpy()))
  testing_labels.append(l.numpy())
  
training_labels_final = np.array(training_labels)     # Convert the list into array
testing_labels_final = np.array(testing_labels)

print(training_labels[0])                             # Print out some data for exploring
print(training_sentences[0])
print(training_labels)
print(training_labels_final)
print(len(training_sentences))

1
b'Clark Gable plays a con man who busts into the life of hard-boiled dame Jean Harlow. He tries to sucker her while she brushes him off with her tough-gal attitude. Despite their cynicism and cons they fall in love. When Gable accidentally kills a man during a sting he runs out leaving loyal Harlow to women\'s prison where she discovers she\'s pregnant. Anita Loos\' and Howard Emmett Rogers\' writing is excellent throughout with many well-drawn and surprising characters (including a Jewish socialist woman inmate and a black woman inmate and her preacher father played with hardly a trace of stereotype). Gable and Harlow show their mettle as actors adding telling nuances and quirks to their characters that send them beyond the typical Gable and Harlow roles. And the direction is much better than you\'d expect from Sam Wood. One beautiful shot has Harlow being inducted into the prison, then led out into a surprisingly snowy courtyard as the camera tracks after her. This is one of the be

In [0]:
########################################### Define the Tokenizer ###########################################
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)

#### a testing code to learn tokenizer ################
tokenizer1 = Tokenizer(num_words = 4, oov_token=oov_tok)       # the use of num_words does not influence fit_on_text to get index
text1 = ['I am trying to testing tokenizer function']          # but limit the output of pad_sequences
tokenizer1.fit_on_texts(text1)
word_index1 = tokenizer1.word_index
sequence1 = tokenizer1.texts_to_sequences(text1)
padded1 = pad_sequences(sequence1,maxlen=10, truncating=trunc_type)
print(word_index1)
print(padded1)

{'<OOV>': 1, 'i': 2, 'am': 3, 'trying': 4, 'to': 5, 'testing': 6, 'tokenizer': 7, 'function': 8}
[[0 0 0 2 3 1 1 1 1 1]]


In [0]:
######################################## Reverse the Tokenizer dictionary ###########################################

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(padded.shape)                                    # num of samples, num of max length
print(padded[1][0:10])                                 # 10 elements from the first padded list
print(len(reverse_word_index))
print(reverse_word_index)                              # reversed dictionary

print(training_sentences[1])
print(decode_review(padded[1]))

total_words = len(reverse_word_index)                  # print '?' when an unknown is encountered
print(reverse_word_index.get(total_words + 1, '?'))    

(25000, 120)
[  59   12    7    2   92 2297   20   11   28  127]
86539
{1: '<OOV>', 2: 'the', 3: 'and', 4: 'a', 5: 'of', 6: 'to', 7: 'is', 8: 'br', 9: 'in', 10: 'it', 11: 'i', 12: 'this', 13: 'that', 14: 'was', 15: 'as', 16: 'for', 17: 'with', 18: 'movie', 19: 'but', 20: 'film', 21: "'s", 22: 'on', 23: 'you', 24: 'not', 25: 'are', 26: 'his', 27: 'he', 28: 'have', 29: 'be', 30: 'one', 31: 'all', 32: 'at', 33: 'by', 34: 'they', 35: 'an', 36: 'who', 37: 'so', 38: 'from', 39: 'like', 40: 'her', 41: "'t", 42: 'or', 43: 'just', 44: 'there', 45: 'about', 46: 'out', 47: "'", 48: 'has', 49: 'if', 50: 'some', 51: 'what', 52: 'good', 53: 'more', 54: 'very', 55: 'when', 56: 'she', 57: 'up', 58: 'can', 59: 'b', 60: 'time', 61: 'no', 62: 'even', 63: 'my', 64: 'would', 65: 'which', 66: 'story', 67: 'only', 68: 'really', 69: 'see', 70: 'their', 71: 'had', 72: 'were', 73: 'me', 74: 'well', 75: 'we', 76: 'than', 77: 'much', 78: 'been', 79: 'get', 80: 'bad', 81: 'will', 82: 'people', 83: 'do', 84: 'also'

In [0]:
########################################### Define and compile the NN ###########################################

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),                                                  # alternative: tf.keras.layers.GlobalAveragePooling1D()
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [0]:
########################################### Model fit both training and testing datasets ###########################################
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


25000/25000 [==============================] - 10s 418us/sample - loss: 0.5047 - acc: 0.7318 - val_loss: 0.3446 - val_acc: 0.8503
Epoch 2/10
25000/25000 [==============================] - 9s 352us/sample - loss: 0.2453 - acc: 0.9066 - val_loss: 0.4024 - val_acc: 0.8208
Epoch 3/10
25000/25000 [==============================] - 9s 351us/sample - loss: 0.0968 - acc: 0.9752 - val_loss: 0.4465 - val_acc: 0.8260
Epoch 4/10
25000/25000 [==============================] - 9s 354us/sample - loss: 0.0238 - acc: 0.9972 - val_loss: 0.5219 - val_acc: 0.8227
Epoch 5/10
25000/25000 [==============================] - 9s 354us/sample - loss: 0.0062 - acc: 0.9995 - val_loss: 0.5852 - val_acc: 0.8224
Epoch 6/10
25000/25000 [==============================] - 9s 360us/sample - loss: 0.0020 - acc: 1.0000 - val_loss: 0.6366 - val_acc: 0.8242
Epoch 7/10
25000/25000 [==============================] - 9s 375us/sample - loss: 0.0011 - acc: 0.9999 - val_loss: 0.6800 - val_acc: 0.8250
Epoch 8/10
25000/25000 [======

In [0]:
########################################### Getting the results of the embeddings layer ###########################################
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape)           # shape: (vocab_size, embedding_dim) they are defined in above cells. Param # is 10000*16

(10000, 16)
10000
86539


In [0]:
########################################### Load the weights into files, can be opened in Google drive ###########################################
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

In [0]:
sentence = ["I really think this is amazing. honest."]
sequence = tokenizer.texts_to_sequences(sentence)
print(sequence)
print(len(sentence))
print(len(sequence))

[[11, 68, 105, 12, 7, 490, 1216]]
1
1
